# Idaho 2008 Presidential Elections: Data Cleaning & Preprocessing

**Goal:** Build a clean, analysis-ready county-level table for Idaho, 2008 by merging the presidential primary and presidential general election results, and then derive summary stats (party totals).

**Output**: A single CSV where each row is a county and columns include:

- Primary per-candidate vote counts (prefixed with `pri_`)
- General per-candidate vote counts (prefixed with `gen_`)
- Party totals: `rep_primary_total`, `dem_primary_total`, `rep_general_total`, `dem_general_total`, `lib_general_total`, `con_general_total`, `ind_general_total`

**Last Updated**: 2025/10/02

## 0. Library Import

In [3]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

/Users/amourtu1934/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1. Inputs & Parameters

Define raw file paths once here so the entire notebook is easy to rerun on another machine. If a path changes, we only update it here. We keep a single `OUTPUT_PATH` so all exports land in one known place.

In [1]:
# ID 2008 dataset path
PRIMARY_PATH = r"../../data/raw/2008/ID/20080527__id__primary__county.csv"
GENERAL_PATH = r"../../data/raw/2008/ID/20081104__id__general__county.csv"

# Output directory
OUTPUT_PATH  = r"../../data/processed/2008/ID/"

# Analysis parameters
DISPLAY_ROWS = 10   # Number of rows to display in dataframes

## 2. Load & Filter

We load primary and general datasets separately and immediately subset to the rows we truly need:

- Restrict `office` to 'President' to avoid mixing down-ballot contests

- Remove columns that are fully missing or irrelevant post-filter (e.g., a district column that’s empty for county-level rows)

### a. Primary Election Dataset

In [4]:
# Load primary data
primary_df = pd.read_csv(PRIMARY_PATH)
primary_df.head(DISPLAY_ROWS)

,county,office,district,party,candidate,votes
0,Ada,President,NaN,Dem.,Hillary Clinton,3634.0
1,Adams,President,NaN,Dem.,Hillary Clinton,55.0
2,Bannock,President,NaN,Dem.,Hillary Clinton,1590.0
3,Bear Lake,President,NaN,Dem.,Hillary Clinton,72.0
4,Benewah,President,NaN,Dem.,Hillary Clinton,224.0
5,Bingham,President,NaN,Dem.,Hillary Clinton,348.0
6,Blaine,President,NaN,Dem.,Hillary Clinton,336.0
7,Boise,President,NaN,Dem.,Hillary Clinton,72.0
8,Bonner,President,NaN,Dem.,Hillary Clinton,551.0
9,Bonneville,President,NaN,Dem.,Hillary Clinton,512.0


In [5]:
# Different values in 'office' column
primary_df["office"].value_counts()

office
U.S. Senate                                  484
State Representative                         260
President                                    220
UNITED STATES REPRESENTATIVE 2nd DISTRICT    220
UNITED STATES REPRESENTATIVE 1st DISTRICT    132
State Senate                                 125
Name: count, dtype: int64

In [6]:
# Only keep rows where 'office' is 'President'
primary_df = primary_df[primary_df["office"] == "President"]
primary_df.shape

(220, 6)

In [7]:
# Now, drop the "office" column as it's no longer needed
# Also, drop the district column
primary_df = primary_df.drop(columns=["office", "district"]).reset_index(drop=True)
primary_df.head(DISPLAY_ROWS)

,county,party,candidate,votes
0,Ada,Dem.,Hillary Clinton,3634.0
1,Adams,Dem.,Hillary Clinton,55.0
2,Bannock,Dem.,Hillary Clinton,1590.0
3,Bear Lake,Dem.,Hillary Clinton,72.0
4,Benewah,Dem.,Hillary Clinton,224.0
5,Bingham,Dem.,Hillary Clinton,348.0
6,Blaine,Dem.,Hillary Clinton,336.0
7,Boise,Dem.,Hillary Clinton,72.0
8,Bonner,Dem.,Hillary Clinton,551.0
9,Bonneville,Dem.,Hillary Clinton,512.0


In [8]:
# Coerce "votes" into integers
primary_df["votes"] = pd.to_numeric(primary_df["votes"], errors="coerce").fillna(0).astype(int)
primary_df.dtypes

county       object
party        object
candidate    object
votes         int64
dtype: object

In [9]:
# Unique parties in primary_df
primary_df["party"].value_counts()

party
Dem.    132
Rep.     88
Name: count, dtype: int64

In [10]:
# Candidates in primary_df
primary_df["candidate"].value_counts()

candidate
Hillary Clinton       44
Keith Russell Judd    44
Barack Obama          44
John McCain           44
Ron Paul              44
Name: count, dtype: int64

In [11]:
# Missing values count
primary_df.isnull().sum()

county       0
party        0
candidate    0
votes        0
dtype: int64

In [24]:
# Calculuate the duplicate rows
duplicate_rows = primary_df.duplicated()
duplicate_rows.sum()

0

In [13]:
# Final look at the cleaned primary_df
primary_df.head(DISPLAY_ROWS)

,county,party,candidate,votes
0,Ada,Dem.,Hillary Clinton,3634
1,Adams,Dem.,Hillary Clinton,55
2,Bannock,Dem.,Hillary Clinton,1590
3,Bear Lake,Dem.,Hillary Clinton,72
4,Benewah,Dem.,Hillary Clinton,224
5,Bingham,Dem.,Hillary Clinton,348
6,Blaine,Dem.,Hillary Clinton,336
7,Boise,Dem.,Hillary Clinton,72
8,Bonner,Dem.,Hillary Clinton,551
9,Bonneville,Dem.,Hillary Clinton,512


In [14]:
# Shape after preprocessing
primary_df.shape

(220, 4)

### b. General Election Dataset

In [15]:
# Load general data
general_df = pd.read_csv(GENERAL_PATH)
general_df.head(DISPLAY_ROWS)

,county,office,district,party,candidate,votes
0,Ada,President,NaN,CON,Chuck Baldwin,777.0
1,Adams,President,NaN,CON,Chuck Baldwin,20.0
2,Bannock,President,NaN,CON,Chuck Baldwin,257.0
3,Bear Lake,President,NaN,CON,Chuck Baldwin,27.0
4,Benewah,President,NaN,CON,Chuck Baldwin,38.0
5,Bingham,President,NaN,CON,Chuck Baldwin,203.0
6,Blaine,President,NaN,CON,Chuck Baldwin,16.0
7,Boise,President,NaN,CON,Chuck Baldwin,25.0
8,Bonner,President,NaN,CON,Chuck Baldwin,195.0
9,Bonneville,President,NaN,CON,Chuck Baldwin,362.0


In [16]:
# Different values in 'office' column
general_df["office"].value_counts()

office
President                                    220
U.S. Senate                                  220
State Representative                         214
State Senate                                 104
UNITED STATES REPRESENTATIVE 1st DISTRICT     88
UNITED STATES REPRESENTATIVE 2nd DISTRICT     88
Name: count, dtype: int64

In [17]:
# Only keep rows where 'office' is 'President'
general_df = general_df[general_df["office"] == "President"]
general_df.shape

(220, 6)

In [18]:
# Now, drop the "office" column as it's no longer needed
# Also, drop the district column
general_df = general_df.drop(columns=["office", "district"]).reset_index(drop=True)
general_df.head(DISPLAY_ROWS)

,county,party,candidate,votes
0,Ada,CON,Chuck Baldwin,777.0
1,Adams,CON,Chuck Baldwin,20.0
2,Bannock,CON,Chuck Baldwin,257.0
3,Bear Lake,CON,Chuck Baldwin,27.0
4,Benewah,CON,Chuck Baldwin,38.0
5,Bingham,CON,Chuck Baldwin,203.0
6,Blaine,CON,Chuck Baldwin,16.0
7,Boise,CON,Chuck Baldwin,25.0
8,Bonner,CON,Chuck Baldwin,195.0
9,Bonneville,CON,Chuck Baldwin,362.0


In [19]:
# Coerce "votes" into integers
general_df["votes"] = pd.to_numeric(general_df["votes"], errors="coerce").fillna(0).astype(int)
general_df.dtypes

county       object
party        object
candidate    object
votes         int64
dtype: object

In [27]:
# Candidates in general_df
general_df["candidate"].value_counts()

candidate
Chuck Baldwin    44
Bob Barr         44
John McCain      44
Ralph Nader      44
Barack Obama     44
Name: count, dtype: int64

In [20]:
# List out all the parties in the general election data
general_df["party"].value_counts()

party
CON    44
LIB    44
REP    44
IND    44
DEM    44
Name: count, dtype: int64

In [21]:
# Missing values count
general_df.isnull().sum()

county       0
party        0
candidate    0
votes        0
dtype: int64

In [22]:
# Final look at the cleaned general_df
general_df.head(DISPLAY_ROWS)

,county,party,candidate,votes
0,Ada,CON,Chuck Baldwin,777
1,Adams,CON,Chuck Baldwin,20
2,Bannock,CON,Chuck Baldwin,257
3,Bear Lake,CON,Chuck Baldwin,27
4,Benewah,CON,Chuck Baldwin,38
5,Bingham,CON,Chuck Baldwin,203
6,Blaine,CON,Chuck Baldwin,16
7,Boise,CON,Chuck Baldwin,25
8,Bonner,CON,Chuck Baldwin,195
9,Bonneville,CON,Chuck Baldwin,362


In [23]:
# Shape after preprocessing
general_df.shape

(220, 4)

## 3. Table Pivoting

We convert tall (one row per county/party/candidate) into wide (one row per county with one column per candidate). This creates the consistent schema with previous group cleaned data.

Helper functions:

- `normalize_party(s)`: in this case, we lower everything so column names are stable with other dataframes
- `candidate_token(name)`: turns “Barack Obama” -> OBAMA, “John McCain” -> MCCAIN, etc. Create a short, readable, unique token for column names
- `pivot_wide(df, prefix, key_col="county")`: Main pivot function
        
    * groups by `county` x `party` × `candidate`, sums `votes`,
    * pivots to columns named like:
        * Primary: `pri_dem_OBAMA`, `pri_rep_MCCAIN`,...
        * General: `gen_dem_OBAMA`, `gen_rep_MCCAIN`,...

    * flattens the MultiIndex into plain column strings,
    * returns one wide row per county

In [39]:
def normalize_party(s: pd.Series) -> pd.Series:
    """
    Normalize party names: Lowercase the three-letter abbreviations
    """
    return(
        s.astype(str)
        .str.strip()
        .str.replace(r"[^\w\s]", "", regex=True)      # drop punctuation
        .str.replace(r"\s+", " ", regex=True)         # collapse spaces
        .str.lower()
    )

In [40]:
SUFFIXES = {
    "JR","SR","JNR","SNR",
    "II","III","IV","V","VI","VII","VIII","IX","X","XI","XII"
}

def candidate_token(name: str) -> str:
    """
    Turn John McCain -> MCCAIN, Barack Obama -> OBAMA
    Skip suffixes, keep last name/token, capitalize, and remove punctuation
    """
    if pd.isna(name):
        return "UNKNOWN"                # Defensive purposes only, would not expect missing values
    
    # Remove suffixes
    raw = str(name).strip()

    # If a comma exists, treat as 'LAST, FIRST ...'
    if "," in raw:
        last_part = raw.split(",", 1)[0]
        last_part = re.sub(r"[^A-Za-z0-9\s]+", "", last_part).strip().upper()
        tokens = last_part.split()
        return tokens[-1] if tokens else "UNKNOWN"

    # Otherwise: remove punctuation, split, then drop trailing suffixes
    tokens = re.sub(r"[^A-Za-z0-9\s]+", "", raw).strip().upper().split()
    while tokens and tokens[-1] in SUFFIXES:
        tokens.pop()
    return tokens[-1] if tokens else "UNKNOWN"

In [41]:
def pivot_wide(df: pd.DataFrame, prefix: str, key_col: str="county") -> pd.DataFrame:
    """
    Pivot the dataframe to wide format based on party and candidate
    """
    # Normalize party names
    df['party_key'] = normalize_party(df['party'])
    
    # Create candidate tokens
    df['candidate_token'] = df['candidate'].apply(candidate_token)
    
    # Create new column names based on party and candidate token
    df['new_col'] = prefix + '_' + df['party'] + '_' + df['candidate_token']
    
    # Pivot the dataframe
    pivot_df = df.pivot_table(index=key_col, 
                              columns=["party_key", "candidate_token"], 
                              values="votes", 
                              aggfunc='sum', 
                              fill_value=0)
    
    # Flatten multi-level columns
    pivot_df.columns = [f"{prefix}_{p}_{c}" for p, c in pivot_df.columns]
    
    # Reset index to turn key_col back into a column
    pivot_df = pivot_df.reset_index()
    
    return pivot_df

In [42]:
# Primary dataframe pivot
primary_pivot = pivot_wide(primary_df, prefix="pri")
primary_pivot.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_JUDD,pri_dem_OBAMA,pri_rep_MCCAIN,pri_rep_PAUL
0,Ada,3634,83,7338,18003,6531
1,Adams,55,1,107,724,304
2,Bannock,1590,115,1937,1935,672
3,Bear Lake,72,7,85,1121,171
4,Benewah,224,17,269,372,199
5,Bingham,348,22,428,4086,864
6,Blaine,336,6,1177,650,109
7,Boise,72,5,155,992,461
8,Bonner,551,17,890,2950,1427
9,Bonneville,512,12,780,4403,1270


In [43]:
# Primary dataframe shape after pivot
primary_pivot.shape

(44, 6)

In [44]:
# General dataframe pivot
general_pivot = pivot_wide(general_df, prefix="gen")
general_pivot.head(DISPLAY_ROWS)

,county,gen_con_BALDWIN,gen_dem_OBAMA,gen_ind_NADER,gen_lib_BARR,gen_rep_MCCAIN
0,Ada,777,82236,1978,1083,93328
1,Adams,20,728,36,19,1517
2,Bannock,257,14792,485,215,19356
3,Bear Lake,27,502,26,11,2377
4,Benewah,38,1407,48,25,2646
5,Bingham,203,4424,207,91,12230
6,Blaine,16,6947,106,63,3439
7,Boise,25,1240,48,27,2433
8,Bonner,195,7840,259,108,11145
9,Bonneville,362,11417,334,256,29334


In [45]:
# General dataframe shape after pivot
general_pivot.shape

(44, 6)

## 4. Merge Dataframes

Before merging, we verify that county names match across primary and general:

In [46]:
# Check if county names match between primary_df and general_df
primary_counties = set(primary_df["county"].unique())
general_counties = set(general_df["county"].unique())
common_counties = primary_counties.intersection(general_counties)
print(f"Number of common counties: {len(common_counties)} out of {len(primary_counties)}")

Number of common counties: 44 out of 44


Great. Since we know that all counties name are matched, we don't need to perform further data preprocessing to match the county names. Thus, we can now merge them:

In [47]:
# Merge primary and general dataframes on 'county'
merged_df = primary_pivot.merge(general_pivot, on="county", how="inner").fillna(0)    # There should be no missing values to fill with 0
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_JUDD,pri_dem_OBAMA,pri_rep_MCCAIN,pri_rep_PAUL,gen_con_BALDWIN,gen_dem_OBAMA,gen_ind_NADER,gen_lib_BARR,gen_rep_MCCAIN
0,Ada,3634,83,7338,18003,6531,777,82236,1978,1083,93328
1,Adams,55,1,107,724,304,20,728,36,19,1517
2,Bannock,1590,115,1937,1935,672,257,14792,485,215,19356
3,Bear Lake,72,7,85,1121,171,27,502,26,11,2377
4,Benewah,224,17,269,372,199,38,1407,48,25,2646
5,Bingham,348,22,428,4086,864,203,4424,207,91,12230
6,Blaine,336,6,1177,650,109,16,6947,106,63,3439
7,Boise,72,5,155,992,461,25,1240,48,27,2433
8,Bonner,551,17,890,2950,1427,195,7840,259,108,11145
9,Bonneville,512,12,780,4403,1270,362,11417,334,256,29334


In [48]:
# Statistics check on merged dataframe 
merged_df.describe()

,pri_dem_CLINTON,pri_dem_JUDD,pri_dem_OBAMA,pri_rep_MCCAIN,pri_rep_PAUL,gen_con_BALDWIN,gen_dem_OBAMA,gen_ind_NADER,gen_lib_BARR,gen_rep_MCCAIN
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,366.409091,16.681818,545.000000,1987.727273,676.931818,107.886364,5373.636364,163.068182,83.136364,9159.363636
std,630.036962,26.243735,1151.431445,2910.360997,1120.124275,156.055628,12890.400990,316.413874,170.932952,15961.860915
min,4.000000,0.000000,8.000000,200.000000,26.000000,1.000000,64.000000,2.000000,1.000000,305.000000
25%,73.000000,4.750000,104.750000,648.250000,192.000000,23.000000,890.000000,40.000000,19.750000,2419.000000
50%,151.000000,7.000000,202.500000,1224.000000,365.000000,40.000000,1558.000000,73.000000,32.500000,3602.000000
75%,339.000000,14.500000,476.500000,1876.500000,594.750000,143.750000,3049.250000,111.250000,64.500000,8126.000000
max,3634.000000,115.000000,7338.000000,18003.000000,6531.000000,777.000000,82236.000000,1978.000000,1083.000000,93328.000000


Now, we will add party totals columns: 

- Primary totals:
    * `rep_primary_total` = sum of all `pri_rep_*` columns
    * `dem_primary_total` = sum of all `pri_dem_*` columns

- General totals:
    * `rep_general_total` = sum of all `gen_rep_*` columns
    * `dem_general_total` = sum of all `gen_dem_*` columns
    * `lib_general_total` = sum of all `gen_lib_*` columns
    * `con_general_total` = sum of all `gen_ccon_*` columns
    * `ind_general_total` = sum of all `gen_ind_*` columns

In [49]:
# Add party totals for primary election
rep_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_rep_")]
dem_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_dem_")]

merged_df["rep_primary_total"] = merged_df[rep_primary_cols].sum(axis=1) if rep_primary_cols else 0
merged_df["dem_primary_total"] = merged_df[dem_primary_cols].sum(axis=1) if dem_primary_cols else 0

In [50]:
# Add party totals for general election
rep_general_cols   = [c for c in merged_df.columns if c.startswith("gen_rep_")]
dem_general_cols   = [c for c in merged_df.columns if c.startswith("gen_dem_")]
lib_general_cols   = [c for c in merged_df.columns if c.startswith("gen_lib_")]
con_general_cols   = [c for c in merged_df.columns if c.startswith("gen_con_")]
ind_general_cols   = [c for c in merged_df.columns if c.startswith("gen_ind_")]

merged_df["rep_general_total"] = merged_df[rep_general_cols].sum(axis=1) if rep_general_cols else 0
merged_df["dem_general_total"] = merged_df[dem_general_cols].sum(axis=1) if dem_general_cols else 0
merged_df["lib_general_total"] = merged_df[lib_general_cols].sum(axis=1) if lib_general_cols else 0
merged_df["con_general_total"] = merged_df[con_general_cols].sum(axis=1) if con_general_cols else 0
merged_df["ind_general_total"] = merged_df[ind_general_cols].sum(axis=1) if ind_general_cols else 0

In [51]:
# Print out all the column names in the final dataframe
print("Final columns in the cleaned dataframe:")
merged_df.columns

Final columns in the cleaned dataframe:


Index(['county', 'pri_dem_CLINTON', 'pri_dem_JUDD', 'pri_dem_OBAMA',
       'pri_rep_MCCAIN', 'pri_rep_PAUL', 'gen_con_BALDWIN', 'gen_dem_OBAMA',
       'gen_ind_NADER', 'gen_lib_BARR', 'gen_rep_MCCAIN', 'rep_primary_total',
       'dem_primary_total', 'rep_general_total', 'dem_general_total',
       'lib_general_total', 'con_general_total', 'ind_general_total'],
      dtype='object')

In [52]:
# Preview merged dataframe with totals
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_JUDD,pri_dem_OBAMA,pri_rep_MCCAIN,pri_rep_PAUL,gen_con_BALDWIN,gen_dem_OBAMA,gen_ind_NADER,gen_lib_BARR,gen_rep_MCCAIN,rep_primary_total,dem_primary_total,rep_general_total,dem_general_total,lib_general_total,con_general_total,ind_general_total
0,Ada,3634,83,7338,18003,6531,777,82236,1978,1083,93328,24534,11055,93328,82236,1083,777,1978
1,Adams,55,1,107,724,304,20,728,36,19,1517,1028,163,1517,728,19,20,36
2,Bannock,1590,115,1937,1935,672,257,14792,485,215,19356,2607,3642,19356,14792,215,257,485
3,Bear Lake,72,7,85,1121,171,27,502,26,11,2377,1292,164,2377,502,11,27,26
4,Benewah,224,17,269,372,199,38,1407,48,25,2646,571,510,2646,1407,25,38,48
5,Bingham,348,22,428,4086,864,203,4424,207,91,12230,4950,798,12230,4424,91,203,207
6,Blaine,336,6,1177,650,109,16,6947,106,63,3439,759,1519,3439,6947,63,16,106
7,Boise,72,5,155,992,461,25,1240,48,27,2433,1453,232,2433,1240,27,25,48
8,Bonner,551,17,890,2950,1427,195,7840,259,108,11145,4377,1458,11145,7840,108,195,259
9,Bonneville,512,12,780,4403,1270,362,11417,334,256,29334,5673,1304,29334,11417,256,362,334


Now, we save the cleaned dataframe into the processed directory.

In [53]:
# Save the cleaned and merged dataframe to CSV
out_dir = Path(OUTPUT_PATH)
out_dir.mkdir(parents=True, exist_ok=True)
merged_df.to_csv(OUTPUT_PATH + "ID.csv", index=False)